In [ ]:
from brepmatching.data import BRepMatchingDataset, load_data
from brepmatching.visualization import show_image, render_predictions
import torch
import os
from torch_geometric.data.batch import Batch
import os
from tqdm import tqdm
from brepmatching.utils import *

In [ ]:
if os.path.exists('../data/geo_test_set.pt'):
    geo_test_graphs = torch.load('../data/geo_test_set.pt')
    topo_test_graphs = torch.load( '../data/topo_test_set.pt')
    both_test_graphs = torch.load( '../data/both_test_set.pt')
else:
    print('Loading Geo')
    geo_cache = torch.load('../data/Geo.pt')
    print('Loading Topo')
    topo_cache = torch.load('../data/Topo.pt')
    print('Loading Both')
    both_cache = torch.load('../data/TopoAndGeo.pt')

    for i in tqdm(range(len(geo_cache['preprocessed_data']))):
        geo_cache['preprocessed_data'][i].__node_sets__.add('bl_overlap_larger_face_percentages')
        geo_cache['preprocessed_data'][i].__node_sets__.add('bl_overlap_larger_edge_percentages')
        geo_cache['preprocessed_data'][i].__node_sets__.add('bl_overlap_smaller_face_percentages')
        geo_cache['preprocessed_data'][i].__node_sets__.add('bl_overlap_smaller_edge_percentages')
    
    for i in tqdm(range(len(topo_cache['preprocessed_data']))):
        topo_cache['preprocessed_data'][i].__node_sets__.add('bl_overlap_larger_face_percentages')
        topo_cache['preprocessed_data'][i].__node_sets__.add('bl_overlap_larger_edge_percentages')
        topo_cache['preprocessed_data'][i].__node_sets__.add('bl_overlap_smaller_face_percentages')
        topo_cache['preprocessed_data'][i].__node_sets__.add('bl_overlap_smaller_edge_percentages')
    
    for i in tqdm(range(len(both_cache['preprocessed_data']))):
        both_cache['preprocessed_data'][i].__node_sets__.add('bl_overlap_larger_face_percentages')
        both_cache['preprocessed_data'][i].__node_sets__.add('bl_overlap_larger_edge_percentages')
        both_cache['preprocessed_data'][i].__node_sets__.add('bl_overlap_smaller_face_percentages')
        both_cache['preprocessed_data'][i].__node_sets__.add('bl_overlap_smaller_edge_percentages')

    ds_geo = BRepMatchingDataset(geo_cache,mode='test',require_onshape_matchings=True)
    ds_topo = BRepMatchingDataset(topo_cache,mode='test',require_onshape_matchings=True)
    ds_both = BRepMatchingDataset(both_cache,mode='test',require_onshape_matchings=True)

    geo_test_graphs = [ds_geo[i] for i in range(len(ds_geo))]
    topo_test_graphs = [ds_topo[i] for i in range(len(ds_topo))]
    both_test_graphs = [ds_both[i] for i in range(len(ds_both))]
    
    torch.save(geo_test_graphs, '../data/geo_test_set.pt')
    torch.save(topo_test_graphs, '../data/topo_test_set.pt')
    torch.save(both_test_graphs, '../data/both_test_set.pt')

In [ ]:
extra_overlaps = []
for i in tqdm(range(len(geo_test_graphs))):
    ex = geo_test_graphs[i]

    overlap_matches = {tuple(x) for x in ex.bl_overlap_faces_matches.T.tolist()}
    onshape_matches = {tuple(x) for x in ex.os_bl_faces_matches.T.tolist()}

    extra_overlaps.append(len(overlap_matches - onshape_matches))

In [ ]:
plt.hist(extra_overlaps)

In [ ]:
ex.bl_overlap_faces_matches

In [ ]:
ex.bl_overlap_smaller_face_percentages

In [ ]:
import meshplot as mp

In [ ]:
ex.left_F_to_faces.shape

In [ ]:
ex.left_V.shape

In [ ]:
mp.plot(ex.left_V.numpy(), ex.left_F.T.numpy(), c =(ex.left_F_to_faces.flatten() == 13).numpy().astype(int), return_plot=True).add_mesh(ex.right_V.numpy() + np.array([.1,0.,0.]), ex.right_F.T.numpy(), c =(ex.right_F_to_faces.flatten() == 8).numpy().astype(int))

In [ ]:
ex

In [ ]:
follow_batch=['left_vertices','right_vertices','left_edges', 'right_edges','left_faces','right_faces', 'faces_matches', 'edges_matches', 'vertices_matches']
geo_batch = Batch.from_data_list(geo_test_graphs,follow_batch=follow_batch)
topo_batch = Batch.from_data_list(topo_test_graphs,follow_batch=follow_batch)
both_batch = Batch.from_data_list(both_test_graphs,follow_batch=follow_batch)

In [ ]:
geo_batch['bl_overlap_faces_matches'][:,geo_batch.bl_overlap_smaller_face_percentages >= .9]

In [ ]:
def compute_metrics_from_batch(batch, small_overlap_thresh=.8, large_overlap_thresh = .0):
    face_exact_metrics = compute_metrics_from_matches(batch, 'faces', batch['bl_exact_faces_matches'])
    edge_exact_metrics = compute_metrics_from_matches(batch, 'edges', batch['bl_exact_edges_matches'])
    vert_exact_metrics = compute_metrics_from_matches(batch, 'vertices', batch['bl_exact_vertices_matches'])

    face_overlap = batch.bl_overlap_faces_matches[:,(batch.bl_overlap_smaller_face_percentages >= small_overlap_thresh) & (batch.bl_overlap_larger_face_percentages >= large_overlap_thresh)]
    edge_overlap = batch.bl_overlap_edges_matches[:,(batch.bl_overlap_smaller_edge_percentages >= small_overlap_thresh) & (batch.bl_overlap_larger_edge_percentages >= large_overlap_thresh)]


    face_coincidence = torch.cat([batch['bl_exact_faces_matches'], face_overlap], dim=1)
    edge_coincidence = torch.cat([batch['bl_exact_edges_matches'], edge_overlap], dim=1)
    vert_coincidence = batch['bl_exact_vertices_matches']

    face_coincidence_metrics = compute_metrics_from_matches(batch, 'faces', face_coincidence)
    edge_coincidence_metrics = compute_metrics_from_matches(batch, 'edges', edge_coincidence)
    vert_coincidence_metrics = compute_metrics_from_matches(batch, 'vertices', vert_coincidence)

    face_onshape_metrics = compute_metrics_from_matches(batch, 'faces', batch['os_bl_faces_matches'])
    edge_onshape_metrics = compute_metrics_from_matches(batch, 'edges', batch['os_bl_edges_matches'])
    vert_onshape_metrics = compute_metrics_from_matches(batch, 'vertices', batch['os_bl_vertices_matches'])

    metrics = {
        'exact': {
            'faces': face_exact_metrics,
            'edges': edge_exact_metrics,
            'vertices': vert_exact_metrics
        },
        'coincidence': {
            'faces': face_coincidence_metrics,
            'edges': edge_coincidence_metrics,
            'vertices': vert_coincidence_metrics
        },
        'onshape': {
            'faces': face_onshape_metrics,
            'edges': edge_onshape_metrics,
            'vertices': vert_onshape_metrics
        }
    }

    return metrics

In [ ]:
all_metrics_99_99 = {
    'geo': compute_metrics_from_batch(geo_batch, .99, .99),
    'topo': compute_metrics_from_batch(topo_batch, .99, .99),
    'both': compute_metrics_from_batch(both_batch, .99, .99)
}

In [ ]:
torch.save(all_metrics, 'baseline_metrics.pt')

In [ ]:

def generate_plots(geo_test_graphs, topo_test_graphs, both_test_graphs, outdir, match_prefix='', title_prefix=''):
    for name,test_set in tqdm((('Geo',geo_test_graphs), ('Topo', topo_test_graphs), ('Both', both_test_graphs)),'Test Sets'):
        follow_batch=['left_vertices','right_vertices','left_edges', 'right_edges','left_faces','right_faces', 'faces_matches', 'edges_matches', 'vertices_matches']
        big_batch = Batch.from_data_list(test_set,follow_batch=follow_batch)
        for topo_type in tqdm(['faces', 'edges', 'vertices'],'Match Sets', leave=False):
            metrics = compute_metrics_from_matches(big_batch, topo_type, big_batch[f'bl_exact_{topo_type}_matches'])
            plot = plot_the_fives(*np.stack([metrics]*2)[:,:-2].T, np.array([0.0,1.0]),f'Exact Matching {name} ({topo_type})')
            outpath = os.path.join(outdir, f'{name}_{topo_type}.png')
            make_containing_dir(outpath)
            plot.savefig(outpath)


In [ ]:
def plot_grid(metrics, method='exact'):
    fig, axes = plt.subplots(3,3, figsize = (24,24))
    for i,ds in enumerate(['geo','topo','both']):
        for j,t in enumerate(['faces','edges','vertices']):
            m = metrics[ds][method][t]
            plot = plot_the_fives(*np.stack([m]*2)[:,:-2].T, np.array([0.0,1.0]),f'{method} -- {ds} ({t})', ax=axes[j,i])
    return fig
coincidence_fig = plot_grid(all_metrics_99_99, 'coincidence')
exact_fig = plot_grid(all_metrics_99_99, 'exact')
onshape_fig = plot_grid(all_metrics_99_99, 'onshape')

coincidence_fig.savefig('coincidence_99_99.png')
exact_fig.savefig('exact_99_99.png')
onshape_fig.savefig('onshape_99_99.png')
